# Intro

This is my progress in the implementation of Hermite mappings for the quadrature representation of cell integrals. See hermite.tm for the math. The notebook [ffc-elements](ffc-elements.ipynb) contains definitions of: *finite element*, *dof* (and its action on functions), *intermediate representation*. There are also [Related commits and PRs](ffc-elements.ipynb#Related-commits-and-PRs).

## Running and debugging

Debugging in notebooks sucks. It's <strike>best</strike> better to use pdb inside emacs:

1. Copy the code to a python file
2. Run using pdb remotely via tramp+anaconda
3. **Remember to set breakpoints in the files inside** `~/local/lib/python2.7/site-packages/` !!!

## Current

* Recall to work both in `quadraturetransformer.py` and `optimisedquadraturetransformer.py`.
* Maybe you have to pass the whole `fiat_element.mapping()` array around?

## Some definitions

**Multiindex: ** An array like `[[0, 0], [0, 1], [1, 0], [1, 1]]`, where each element is a list of derivatives and each element in a derivative is the index of the spatial dimension along which it is taken. In the example we have $\partial^2_x , \partial_x \partial_y , \partial_y \partial_x $ and $ \partial^2_y$

In [ ]:
from __future__ import print_function
from dolfin import *
import ffc
from ffc.log import add_logfile, set_level, DEBUG

set_level(DEBUG)
add_logfile("/tmp/fenics.log")

# FFC  form compilation

The following snippet produces `/tmp/biharmonic.h`: a c++ header with code which is later wrapped into a python API for using the bilinear form

$$ \int \Delta u \Delta v \ \mathrm{d}x$$

It contains classes with:

* Finite elements.
* Local to global DOF maps.
* **Cell integral computation:** `class bh_cell_integral_0_otherwise: public ufc::cell_integral` This is what interests us now.
* The form itself.

In [ ]:
mesh = UnitSquareMesh(32, 32)
V = FunctionSpace(mesh, "Hermite", 3)
u = TrialFunction(V)
v = TestFunction(V)
#a = inner(div(grad(u)), div(grad(v)))*dx
#with open("/tmp/biharmonic.h", "wt") as f:
#    out = ffc.compile_form(a, prefix="bh")
#    #print(out[1])
#    f.write(out[0])
F = div(avg(grad(v)))*dS
#F = div(grad(v))*dx
with open("/tmp/laplace.h", "wt") as f:
    out = ffc.compile_form(F, prefix="la", parameters={'representation': 'quadrature',
                                                       'quadrature_degree': 1})
    f.write(out[0])

# Integration of expressions



In [ ]:
import numpy as np

grid_sizes = [32, 64, 128, 256]
p_vals = range(1, 4)
results_her = np.zeros(len(grid_sizes))
results_lag = np.zeros((len(grid_sizes), len(p_vals)))

for i, n in enumerate(grid_sizes):
    mesh = UnitSquareMesh(n, n)
    V = FunctionSpace(mesh, "Hermite", 3)
    W = [None] + [FunctionSpace(mesh, "Lagrange", p) for p in p_vals]
    f = Expression("x[0]*x[0]*x[1]*x[1]", degree=3)
    v = project(f, V)
    w = [None] + [project(f, W[p]) for p in p_vals]
    exact = 1./9.
    results_her[i] = assemble(v*dx)
    results_lag[i] = np.array([assemble(w[p]*dx) for p in p_vals])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pl

pl.figure(figsize=(10, 10))
pl.plot(grid_sizes, np.log(results_her - exact), label="H3")
for p in p_vals:
    pl.plot(grid_sizes, np.log(results_lag[:,p-1] - exact), label="L%d" % p)
pl.title("Log error")
pl.legend()
_ = pl.show()